# Scraping Data

In [1]:
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm # make your loops show a progress meter
import warnings
warnings.filterwarnings('ignore')
url = "https://api.youtubemultidownloader.com/playlist"
payload = ""
s = requests.Session()

## Single Page (Upto 200 videos)

In [2]:
plurl = "https://youtube.com/playlist?list=PLJCXrsW_esBlb1BYfeNhY4kJQuNYoDpF_" #148
plurl = "https://www.youtube.com/playlist?list=PLMOobVGrchXPeBgrrnQN6pimrmpTsWFiZ" #python code optimization
plurl = "https://youtube.com/playlist?list=PLZDMziPRHWIrkfzKIXUOrir_gwD0cD9f8" #test

In [3]:
querystring = {"url":plurl,"nextPageToken":""}
pl_json_data = requests.request("GET", url, data=payload, params=querystring).json()

In [4]:
errurls = []
df = pd.DataFrame(columns=['id', 'height', 'width', 'size', 'ext', 'url'])
for i in tqdm(range(pl_json_data['totalResults']), total = pl_json_data['totalResults']):
  vid = pl_json_data['items'][i]['id']
  vapiurl = f"https://api.youtubemultidownloader.com/video?url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3D{vid}"
  v_json_data = s.get(vapiurl).json()
  if v_json_data['status'] == 0 or v_json_data['format'] == None: 
      errurls.append(pl_json_data['items'][i]['url'])
      continue
  temp_df = pd.DataFrame(v_json_data['format'])
  temp_df["vid"] = vid
  temp_df["title"] = v_json_data["title"]
  df = df.append(temp_df)
df = df.iloc[:,[0,1,2,3,4,6,7,5]]
df['id'] = [int(x) for x in df['id']]

print(df['title'].nunique())
print(df['title'].unique())
df

  0%|          | 0/8 [00:00<?, ?it/s]

7
['4 second video'
 'COUNTDOWN TIMER ( v 679 ) 30 sec with sound music effects 4K'
 'New Coke - A Complete Disaster?'
 'Web Scraping with Python - Beautiful Soup Crash Course' '10 hour timer'
 '1 Week 168 Hour Timer Countdown with Alarm Sound / 168 H / 168 Hrs - Longest Video on YouTube'
 '200 Hour Countdown Timer Video – Longest Timer in the World on Youtube -']


,id,height,width,size,ext,vid,title,url
0,137,1080.0,1920.0,263890,mp4,FtdpDjV0dMY,4 second video,https://redirector.googlevideo.com/videoplayba...
1,248,1080.0,1920.0,148307,webm,FtdpDjV0dMY,4 second video,https://redirector.googlevideo.com/videoplayba...
2,22,720.0,1280.0,0,mp4,FtdpDjV0dMY,4 second video,https://redirector.googlevideo.com/videoplayba...
3,136,720.0,1280.0,150157,mp4,FtdpDjV0dMY,4 second video,https://redirector.googlevideo.com/videoplayba...
4,247,720.0,1280.0,104000,webm,FtdpDjV0dMY,4 second video,https://redirector.googlevideo.com/videoplayba...
...,...,...,...,...,...,...,...,...
2,139,NaN,NaN,4389847041,m4a,mivcDoTjpQE,200 Hour Countdown Timer Video – Longest Timer...,https://redirector.googlevideo.com/videoplayba...
3,140,NaN,NaN,11651866098,m4a,mivcDoTjpQE,200 Hour Countdown Timer Video – Longest Timer...,https://redirector.googlevideo.com/videoplayba...
4,249,NaN,NaN,326225683,webm,mivcDoTjpQE,200 Hour Countdown Timer Video – Longest Timer...,https://redirector.googlevideo.com/videoplayba...
5,250,NaN,NaN,326225683,webm,mivcDoTjpQE,200 Hour Countdown Timer Video – Longest Timer...,https://redirector.googlevideo.com/videoplayba...


In [5]:
len(errurls)

1

In [6]:
errurls

['https://www.youtube.com/watch?v=QvW61K2s0tA']

## Multipage

In [7]:
plurl = "https://www.youtube.com/playlist?list=PLFsQleAWXsj_4yDeebiIADdH5FMayBiJo" #310

In [8]:
errurls = []
df = pd.DataFrame(columns=['id', 'height', 'width', 'size', 'ext', 'url'])
pl_json_data = dict()
pl_json_data['nextPageToken'] = ""
while 1:
  querystring = {"url":plurl,"nextPageToken":pl_json_data['nextPageToken']}
  pl_json_data = requests.request("GET", url, data=payload, params=querystring).json()
  for i in tqdm(range(pl_json_data['resultsPerPage']), total = pl_json_data['resultsPerPage']):
    vid = pl_json_data['items'][i]['id']
    vapiurl = f"https://api.youtubemultidownloader.com/video?url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3D{vid}"
    v_json_data = s.get(vapiurl).json()
    if v_json_data['status'] == 0 or v_json_data['format'] == None: 
        errurls.append(pl_json_data['items'][i]['url'])
        continue
    temp_df = pd.DataFrame(v_json_data['format'])
    temp_df["vid"] = vid
    temp_df["title"] = v_json_data["title"]
    df = df.append(temp_df)
  if pl_json_data['nextPageToken'] == None : break
df = df.iloc[:,[0,1,2,3,4,6,7,5]]
df['id'] = [int(x) for x in df['id']]

print(df['title'].nunique())
print(df['title'].unique())
df

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

252
['Yee' 'color red' 'Terrible Mall Commercial'
 '[HD] Sandra Annenberg Cai Ao Vivo no Programa Como Será'
 'name a yellow fruit' "Nate's Firepole Fail"
 'Guy takes video instead of picture.'
 'School of Rock Hawaii 5-0 Drum Fill' 'Trick Shot' 'Kiss'
 "Flutterbye fairy toy flies into fire O'Fortuna" 'burp helium (so funny)'
 'You schleep bro   yeeeaaaah  he schleep  [REUPLOADED]' 'Spiderman'
 'My Cat is Crazy' 'ajrojrojro' 'Bobby'
 'Seth Rogen Laugh + Cantina Song = Nightmares'
 'How to light a firecracker' 'moshfit'
 "I'm from Winnipeg You Idiot! Jericho Owns the World !!"
 'Professor Scott Steiner Typography' 'HERE I COME I AM CINNAMON'
 "Не гладь меня! Don't touch me" 'GoPro Test / Welcome To My Channel'
 'Flying nut high five' 'Barney Meets His Maker'
 'Turn that Poop into Wine' 'Wow a Yard Sale!' 'Super Марио Brothers'
 'I Just Found 12 Bricks' 'Kimchi slap' '드라마 레전드급 무리수ㅋㅋㅋ' 'wowww'
 'Airbus A320 test flight gone wrong' 'Congratulations!!!!'
 'Kaito Cat Hates the Vet' 'Birthday

,id,height,width,size,ext,vid,title,url
0,18,360.0,480.0,0,mp4,q6EoRBvdVPQ,Yee,https://redirector.googlevideo.com/videoplayba...
1,134,360.0,480.0,57150,mp4,q6EoRBvdVPQ,Yee,https://redirector.googlevideo.com/videoplayba...
2,243,360.0,480.0,104373,webm,q6EoRBvdVPQ,Yee,https://redirector.googlevideo.com/videoplayba...
3,396,360.0,480.0,78099,mp4,q6EoRBvdVPQ,Yee,https://redirector.googlevideo.com/videoplayba...
4,133,240.0,320.0,33314,mp4,q6EoRBvdVPQ,Yee,https://redirector.googlevideo.com/videoplayba...
...,...,...,...,...,...,...,...,...
11,139,NaN,NaN,2082164,m4a,wRRsXxE1KVY,John Cena Prank Call,https://redirector.googlevideo.com/videoplayba...
12,140,NaN,NaN,5556917,m4a,wRRsXxE1KVY,John Cena Prank Call,https://redirector.googlevideo.com/videoplayba...
13,249,NaN,NaN,2141847,webm,wRRsXxE1KVY,John Cena Prank Call,https://redirector.googlevideo.com/videoplayba...
14,250,NaN,NaN,2786575,webm,wRRsXxE1KVY,John Cena Prank Call,https://redirector.googlevideo.com/videoplayba...


In [9]:
len(errurls)

58

In [10]:
errurls

['https://www.youtube.com/watch?v=ixQkcuZhXg8',
 'https://www.youtube.com/watch?v=EWF8Nfm-LLk',
 'https://www.youtube.com/watch?v=91UczWPzMjg',
 'https://www.youtube.com/watch?v=v3i8vsIUA7Q',
 'https://www.youtube.com/watch?v=wtUMQZqI3x0',
 'https://www.youtube.com/watch?v=lg_sd_EpTDs',
 'https://www.youtube.com/watch?v=ssvkQ6HMlmQ',
 'https://www.youtube.com/watch?v=sz2mmM-kN1I',
 'https://www.youtube.com/watch?v=xLkiN9K0GR0',
 'https://www.youtube.com/watch?v=FahK9srJq10',
 'https://www.youtube.com/watch?v=XYQnkBkwnFE',
 'https://www.youtube.com/watch?v=NO7N_zRRPo8',
 'https://www.youtube.com/watch?v=4YqjCAWD2W4',
 'https://www.youtube.com/watch?v=2tgU4dOEcrs',
 'https://www.youtube.com/watch?v=L5Z4OH6sj1c',
 'https://www.youtube.com/watch?v=Q67SozSkzuU',
 'https://www.youtube.com/watch?v=oeb5LdAyLC8',
 'https://www.youtube.com/watch?v=liXGKC3kMfM',
 'https://www.youtube.com/watch?v=dvyZfa9x3UU',
 'https://www.youtube.com/watch?v=61JZYH5JHV8',
 'https://www.youtube.com/watch?v=VC-Hjn

# Extracting Required Links

In [11]:
download_type = ["video", "audio"]
video_quality = ['240p', '360p', '480p', '720p', '720p60', '1080p', '1080p60', '1440p', '1440p60', '2160p', '2160p60']
download_id = []

In [12]:
#   id    height    ext   
#  ----- --------- ------ 
#   18    240       mp4   
#   133   240       mp4   
#   394   240       mp4   
#   395   240       mp4   
#   18    360       mp4   
#   134   360       mp4   
#   396   360       mp4   
#   135   480       mp4   
#   397   480       mp4   
#   22    720       mp4   
#   136   720       mp4   
#   298   720       mp4   
#   398   720       mp4   
#   137   1080      mp4   
#   299   1080      mp4   
#   399   1080      mp4   
#   22    1280      mp4   
#   136   1280      mp4   
#   398   1280      mp4   
#   137   1920      mp4   
#   399   1920      mp4
#   400   1440      mp4   
#   401   2160      mp4  


#   139             m4a   
#   140             m4a  


#   mp3128 128kbps	mp3
#   auto   auto     mp4

In [13]:
print(json.dumps(v_json_data, indent=2))

{
  "url": "https://www.youtube.com/watch?v=wRRsXxE1KVY",
  "quality": [
    "360p",
    "240p",
    "144p"
  ],
  "subtitle": null,
  "title": "John Cena Prank Call",
  "description": "One of the best John Cena prank call ever. Been so viral that many media houses have covered it in their shows.\nThank you for the support.\n\n\n\nMusic Copyrights: WWE",
  "thumbnails": "https://i.ytimg.com/vi/wRRsXxE1KVY/default.jpg",
  "duration": 350,
  "preview": "https://www.youtube.com/embed/wRRsXxE1KVY?rel=0&autoplay=1",
  "audioOnly": false,
  "responseTime": 0.542,
  "needProxy": true,
  "isProtected": false,
  "copyright": null,
  "license": [],
  "status": true,
  "format": [
    {
      "id": "18",
      "height": 360,
      "width": 640,
      "size": 0,
      "ext": "mp4",
      "url": "https://redirector.googlevideo.com/videoplayback?api=youtubemultidownloader.com&expire=1675712294&ei=xgLhY_blB4O18gONy6nYBA&ip=2a03%3Ab0c0%3A3%3Ae0%3A%3Ae6%3Af001&id=o-AKaoYe-6oxCDL1A3Qu-2kPEAUDaVpr9telRiX

In [14]:
print(json.dumps(pl_json_data, indent=2))

{
  "items": [
    {
      "id": "-m-vVKHideI",
      "thumbnails": "https://i.ytimg.com/vi/-m-vVKHideI/default.jpg",
      "title": "Paranormal Pugtivity",
      "preview": "https://www.youtube.com/embed/-m-vVKHideI?rel=0&autoplay=1",
      "duration": 21,
      "url": "https://www.youtube.com/watch?v=-m-vVKHideI"
    },
    {
      "id": "-OTPEyj5l3c",
      "thumbnails": "https://i.ytimg.com/vi/-OTPEyj5l3c/default.jpg",
      "title": "Cannoli murder",
      "preview": "https://www.youtube.com/embed/-OTPEyj5l3c?rel=0&autoplay=1",
      "duration": 5,
      "url": "https://www.youtube.com/watch?v=-OTPEyj5l3c"
    },
    {
      "id": "wgTZUU4gJ2U",
      "thumbnails": "https://i.ytimg.com/vi/wgTZUU4gJ2U/default.jpg",
      "title": "real killer of jfk found [uncovered]",
      "preview": "https://www.youtube.com/embed/wgTZUU4gJ2U?rel=0&autoplay=1",
      "duration": 42,
      "url": "https://www.youtube.com/watch?v=wgTZUU4gJ2U"
    },
    {
      "id": "-VMTYIc38DE",
      "thumbnail